![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/quickstart-ci/ClassificationWithAutomatedML.png)

# Quickstart: Fraud Classification using Automated ML

In this quickstart, you use automated machine learning in Azure Machine Learning service to  train a classification model on an associated fraud credit card dataset. This process accepts training data and configuration settings, and automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model.

You will learn how to:

> * Download a dataset and look at the data
> * Train a machine learning classification model using autoML 
> * Explore the results


### Connect to your workspace and create an experiment

You start with importing some libraries and creating an experiment to track the runs in your workspace. A workspace can have multiple experiments, and all the users that have access to the workspace can collaborate on them. 

In [4]:
!pip install azureml-train automl

     |████████████████████████████████| 71 kB 82 kB/s  eta 0:00:01
     |████████████████████████████████| 76 kB 4.8 MB/s  eta 0:00:01
     |████████████████████████████████| 24.9 MB 255 kB/s eta 0:00:01
     |████████████████████████████████| 150 kB 22.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 13.4 MB/s eta 0:00:01
     |████████████████████████████████| 10.4 MB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 627 kB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 1.2 MB/s  eta 0:00:01
     |████████████████████████████████| 157 kB 15.2 MB/s eta 0:00:01
  Created wheel for sklearn-deap2: filename=sklearn_deap2-0.2.2-py3-none-any.whl size=9066 sha256=9a52013e59aae6b65b66004adb42f8cfda69ce9bc300ca89b69d146289e3e41f
  Stored in directory: /home/alibina/.cache/pip/wheels/29/d4/25/6d39525150f3e81c3a8d5dd4f7694c3047470c3c6fc7e70bfc
  Created wheel for dill: filename=d

In [6]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

ModuleNotFoundError: No module named 'azureml.train.automl'

In [ ]:
ws = Workspace.from_config()

# choose a name for your experiment
experiment_name = "fraud-classification-automl-tutorial"

experiment = Experiment(ws, experiment_name)

### Load Data

Load the credit card dataset from a csv file containing both training features and labels. The features are inputs to the model, while the training labels represent the expected output of the model. Next, we'll split the data using random_split and extract the training data for the model.


Follow this [how-to](https://aka.ms/azureml/howto/createdatasets) if you want to learn more about Datasets and how to use them.




In [ ]:
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/creditcard.csv"
dataset = Dataset.Tabular.from_delimited_files(data)
training_data, validation_data = dataset.random_split(percentage=0.8, seed=223)
label_column_name = "Class"

## Train



When you use automated machine learning in Azure ML, you input training data and configuration settings, and the process automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model. 
Learn more about how you configure automated ML [here](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train).


Instantiate an [AutoMLConfig](https://docs.microsoft.com/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py) object. This defines the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. 
|**enable_early_stopping**  | Stop the run if the metric score is not showing improvement.|
|**n_cross_validations**|Number of cross validation splits.|
|**training_data**|Input dataset, containing both features and label column.|
|**label_column_name**|The name of the label column.|

You can find more information about primary metrics [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-configure-auto-train#primary-metric)

In [ ]:
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": "average_precision_score_weighted",
    "experiment_timeout_hours": 0.25,  # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ability to find the best model possible
    "verbosity": logging.INFO,
    "enable_stack_ensemble": False,
}

automl_config = AutoMLConfig(
    task="classification",
    debug_log="automl_errors.log",
    training_data=training_data,
    label_column_name=label_column_name,
    **automl_settings,
)

Call the `submit` method on the experiment object and pass the run configuration. 

**Note: Depending on the data and the number of iterations an AutoML run can take a while to complete.**

In this example, we specify `show_output = True` to print currently running iterations to the console. It is also possible to navigate to the experiment through the **Experiment** activity tab in the left menu, and monitor the run status from there.

In [ ]:
local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
local_run

### Analyze results

Below we select the best model from our iterations. The `get_output` method on `automl_classifier` returns the best run and the model for the run.

In [ ]:
best_run, best_model = local_run.get_output()
best_model

## Tests

Now that the model is trained, split the data in the same way the data was split for training (The difference here is the data is being split locally) and then run the test data through the trained model to get the predicted values.

In [ ]:
# convert the test data to dataframe
X_test_df = validation_data.drop_columns(
    columns=[label_column_name]
).to_pandas_dataframe()
y_test_df = validation_data.keep_columns(
    columns=[label_column_name], validate=True
).to_pandas_dataframe()

In [ ]:
# call the predict functions on the model
y_pred = best_model.predict(X_test_df)
y_pred



### Calculate metrics for the prediction

Now visualize the data to show what our truth (actual) values are compared to the predicted values 
from the trained model that was returned.



In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools

cf = confusion_matrix(y_test_df.values, y_pred)
plt.imshow(cf, cmap=plt.cm.Blues, interpolation="nearest")
plt.colorbar()
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
class_labels = ["False", "True"]
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels)
plt.yticks([-0.5, 0, 1, 1.5], ["", "False", "True", ""])
# plotting text value inside cells
thresh = cf.max() / 2.0
for i, j in itertools.product(range(cf.shape[0]), range(cf.shape[1])):
    plt.text(
        j,
        i,
        format(cf[i, j], "d"),
        horizontalalignment="center",
        color="white" if cf[i, j] > thresh else "black",
    )
plt.show()

## Control cost and further exploration

If you want to control cost you can stop the compute instance this notebook is running on by clicking the "Stop compute" button next to the status dropdown in the menu above.


If you want to run more notebook samples, you can click on **Sample Notebooks** next to the **Files** view and explore the notebooks made available for you there.